## Lesson 11 - Import, LOAD CSV,  Upgrade database and more Cypher





### Table of Contents

* [Neo4j-Admin Import Standalone Server](#neo4j-admin)
* [Import CSV by Cypher (in Neo4j Browser)](#ImportCSVCypher)
* [Import CSV by Cypher shell](#ImportCSVShell)
* [Bulk Import CSV](#BulkImportCSV)
* [Import CSV and Create Relationships](#ImportNodeRelationshipsCypher)
* [Recommendations – Movie Database](#MovieRecommendations)




<a id="neo4j-admin"></a>
## Neo4j-Admin Import Standalone Server

Run the Neo4j-Admin import to create the database. In this example, we are running a bulk import into a new database called dataload1. Node labels are STORE.

```
 ./bin/neo4j-admin import --database=dataload1 --nodes=:STORE="/home/ubuntu/tmp/header.csv,/home/ubuntu/tmp/nodes.csv" --skip-duplicate-nodes=true --high-io=true
```

### Log into Neo4j under Cypher Shell

```
:use system
create database dataload1
show databases
:use dataload1
MATCH (n:STORE) return n limit 3;
```

### Run a second Neo4j-Admin import to create another database

In this example, we are running a bulk import into a new database called dataload2. Node labels are STORE.

```
./bin/neo4j-admin import --database=dataload2 --nodes=:STORE="/home/ubuntu/tmp/header.csv,/home/ubuntu/tmp/nodes.csv" --skip-duplicate-nodes=true --high-io=true
```

```
:use system
create database dataload2
show databases
:use dataload2
MATCH (n:STORE) return n limit 3;
```

### Neo4j-Admin Import Cluster

Run the Neo4j-Admin import on each cluster member to create the database. In this example, we are running a bulk import into a new database called dataload1. Node labels are STORE

```
./bin/neo4j-admin import --database=dataload1 --nodes=:STORE="/home/ubuntu/tmp/header.csv,/home/ubuntu/tmp/nodes.csv" --skip-duplicate-nodes=true --high-io=true
```

Log into the Neo4j 4.0 Cluster Leader. Once the database is created on the cluster leader, it will be propagated to the other cluster members.

```
:use system
create database dataload1
show databases
:use dataload1
MATCH (n:STORE) return n limit 3;
```

### Neo4j-Admin Restore

In this example, we will backup the dataload1 database and use it to create a new dataload3 database.

```
./bin/neo4j-admin backup --backup-dir=/home/ubuntu/tmp/backups --database=dataload1
 ./bin/neo4j-admin restore --from=/home/ubuntu/tmp/backups/dataload1 --database=dataload3
```

If you want to replace an existing 4.0 database using neo4j-admin restore, you would run the following:

```
:use system
stop database dataload2
```

```
./bin/neo4j-admin restore --from=/home/ubuntu/tmp/backups/dataload1 --database=dataload2 --force
```

```
start database dataload2
MATCH (n:STORE) return n limit 3;
```

<a id="ImportCSVCypher"></a>
## Import CSV by Cypher (in Neo4j Browser)

Neo4j 預設情況下，基於安全性的考量，要匯入的 csv 檔案必須先放到該資料庫的 Import Folder，每個資料庫的位置不同，以系統取得的某個資料庫的 Import Folder 如下：

Windows OS
```python
C:\Users\davidlanz\AppData\Local\Neo4j\Relate\Data\dbmss\dbms-968206b1-4a6f-4b9d-af79-0901e1e819a6
```

Mac OS
```python
/Users/davidlanz/Library/Application Support/com.Neo4j.Relate/Data/dbmss/dbms-5aab44e4-5d1d-4b91-94ef-84a7653d534f/import
```

查詢 Import Folder 的步驟，先點選資料庫的 Manage

<img src="images/ManageDatabase.png">

在 Open Folder 中選擇 Import 即可

<img src="images/ImportFromManageDashboard.png">

### 準備 movie.csv

路徑: ./csv/sample/movie.csv

```
id,title,country,year
1,Wall Street,USA,1987
2,The American President,USA,1995
3,The Shawshank Redemption,USA,1994
```

把 CSV 檔案複製到 Import Folder 之後就可以開始操作了，

<img src="images/open_import_folder.png" width=800>

檔案都必須以 file:/// 作為開頭，以下直接提供範例

從 movies.csv 讀入資料，一次讀取一行，並列出前面十行的內容。這部分只是讀取 CSV，並沒有建立或匯入任何圖形資料庫。

```
LOAD CSV FROM "file:///movies.csv" AS line
RETURN line
LIMIT 10
```

<img src="images/LOAD_CSV_1.png" width=800>

CSV 也可以是網路位置

```
LOAD CSV FROM "http://www.somewhere/movies.csv" AS line
RETURN line
LIMIT 10
```

從 movies.csv 讀入資料，一次讀取一行，並將第一欄作為 title 屬性，第二欄作為 released 屬性建立 Movie 節點。

```
LOAD CSV FROM "file:///movies.csv" AS line
CREATE (:Movie { title: line[0], released: line[1]})
```

與前例相同，差別是有引用 CSV Header，語法更直覺。另外，也可以自訂 CSV 分隔符號。

<a id="ImportCSVShell"></a>
## Import CSV by Cypher shell

不過實務上，匯入 CSV 如果會是個定期作業，當然就無法總是透過 Neo4j Browser 操作，建議用 cypher-shell 命令列工具會更有彈性，以下舉例，暫不做深入介紹。

```
bin/cypher-shell -u neo4j -p neo4j 'LOAD CSV FROM "file:///movies.csv" AS line RETURN line LIMIT 5'
```

<a id="BulkImportCSV"></a>
## Bulk Import CSV

如果要匯入的 CSV 資料量相當大，建議多加上 USING PERIODIC COMMIT 敘述，讓 Neo4j 每匯入一定數量的資料行之後就 commit，避免在交易過程中耗費大量記憶體。因為整個 LOAD CSV 預設就是一筆交易，只有全部成功或是全部失敗，但 CSV 檔案太大就會導致 Neo4j 一直在讀取而無法完成交易，最後記憶體就會耗盡。

```
:auto USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///movies.csv" AS line
CREATE (:Movie { title: line.title, released: line.released})
```

PERIODIC COMMIT 預設是每 1000 筆就 commit一次，你可以修改預設值如下，改為 2000 筆 commit 一次。

```
:auto USING PERIODIC COMMIT 2000
LOAD CSV WITH HEADERS FROM "file:///movies.csv" AS line
CREATE (:Movie { title: line.title, released: line.released})
```

<a id="ImportNodeRelationshipsCypher"></a>
## Import CSV and Create Relationships



### 準備 movie.csv

路徑: ./csv/sample/movie.csv

```
id,title,country,year
1,Wall Street,USA,1987
2,The American President,USA,1995
3,The Shawshank Redemption,USA,1994
```

### 準備  persons.csv

路徑: ./csv/sample/persons.csv

```
id,name
1,Charlie Sheen
2,Michael Douglas
3,Martin Sheen
4,Morgan Freeman
```

### 準備  roles.csv

路徑: ./csv/sample/persons.csv

```
personId,movieId,role
1,1,Bud Fox
4,1,Carl Fox
3,1,Gordon Gekko
4,2,A.J. MacInerney
3,2,President Andrew Shepherd
5,3,Ellis Boyd 'Red' Redding
```

Before starting our imports, we will prepare our database by creating indexes and constraints. Since we expect the id property on Person and Movie to be unique in each set, we will create a unique constraint. This protects us from invalid data since constraint creation will fail if there are multiple nodes with the same id property.

Creating a unique constraint also implicitly creates a unique index. The id property is a temporary property used to look up the appropriate nodes for a relationship when importing the third file. By indexing the id property, node lookup (e.g. by MATCH) will be much faster.

### Create CONSTRAINT on Person
```
CREATE CONSTRAINT personIdConstraint ON (person:Person) ASSERT person.id IS UNIQUE
```

<img src="images/create_constraint_person.png">

### Create CONSTRAINT on Movie

```
CREATE CONSTRAINT movieIdConstraint ON (movie:Movie) ASSERT movie.id IS UNIQUE
```

<img src="images/create_constraint_movie.png">

### Create INDEX for Country

```
CREATE INDEX FOR (c:Country) ON (c.name)
```

<img src="images/create_index_by_country_name.png">

### Copy  movie.csv, persons.csv and roles.csv to import folder

<img src="images/copy_csv_to_import_folder_person.png">

In this example, the CSV files are stored in the default import directory on the database server, and we can access them using a file:/// URL. Other locations are configurable, and additionally, LOAD CSV supports accessing CSV files via HTTPS, HTTP, and FTP. For complete instructions, see Cypher Manual → LOAD CSV.

Using the following Cypher queries, we will create a node for each person, a node for each movie and a relationship between the two with a property denoting the role. We are also keeping track of the country in which each movie was made.

Let’s start with importing the persons.csv file. Here is the Cypher used to do the import:

### LOAD CSV with Persons, create Person nodes
```
LOAD CSV WITH HEADERS FROM "file:///persons.csv" AS csvLine
CREATE (p:Person {id: toInteger(csvLine.id), name: csvLine.name})
```

<img src="images/load_csv_person.png">

Now, let’s import the movies. This time, we are also creating a relationship to the country in which the movie was made. We are using MERGE to create nodes that represent countries. Using MERGE avoids creating duplicate country nodes in the case where multiple movies have been made in the same country.

### LOAD CSV with Movies, create Movie nodes

```
LOAD CSV WITH HEADERS FROM "file:///movies.csv" AS csvLine
MERGE (country:Country {name: csvLine.country})
CREATE (movie:Movie {id: toInteger(csvLine.id), title: csvLine.title, year:toInteger(csvLine.year)})
CREATE (movie)-[:MADE_IN]->(country)
```

<img src="images/load_csv_movies.png">

Finally, we will create relationships between the persons and the movies; one actor can participate in many movies, and one movie has many actors in it. Now importing the relationships is a matter of finding the nodes and then creating relationships between them.

### LOAD CSV with Roles, create  relationships between the persons and the movies

```
:auto USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS FROM "file:///roles.csv" AS csvLine
MATCH (person:Person {id: toInteger(csvLine.personId)}),(movie:Movie {id: toInteger(csvLine.movieId)})
CREATE (person)-[:PLAYED {role: csvLine.role}]->(movie)
```

<img src="images/load_csv_roles_periodic.png">

For larger data files, it is useful to use the hint USING PERIODIC COMMIT clause of LOAD CSV. This hint tells Neo4j that the query might build up inordinate amounts of transaction state, and so needs to be periodically committed. For more information, 

Finally, since the id property was only necessary to import the relationships, we can drop the constraints and the id property from all movie and person nodes.

### Cypher query
```
MATCH (n)-[r]-(m) RETURN *
```

<img src="images/cypher_match_when_load_csv.png">

<img src="images/person_movie_country.png">

<img src="images/load_csv_relationships_flowchart.png">

We can delete CONSTRAINT by using Cypher query down below, BUT not now (以下 Cypher 請不要現在執行)

```
DROP CONSTRAINT personIdConstraint
```

```
DROP CONSTRAINT movieIdConstraint
```

Or Delete id on each node:

```
MATCH (n)
WHERE n:Person OR n:Movie
REMOVE n.id
```

<a id="MovieRecommendations"></a>
## Recommendations – Movie Database

https://www.dropbox.com/s/aktlosj8bvd579e/recommendations.db.zip?dl=0

### 解壓縮 graph.db，複製資料夾至新建資料庫的 databases 資料夾
<img src="images/restore_upgrade_old_graphdb.png">

### 修改設定(settings)中的設定升級為true
<img src="images/neo4j_settings_upgrade_true.png">

### General search
```
MATCH (n)-[r]-(m) RETURN * LIMIT 25
```

<img src="images/movie recommendation_match_all.png">

### Query Movie title: Titanic (鐵達尼號)
```
MATCH p=(n:Movie {title: "Titanic"})-[:DIRECTED|ACTED_IN|IN_GENRE]-(m) RETURN p LIMIT 25
```
<img src="images/cypher_movie_titanic_relationships.png">

### Query Movie title: Inception (全面啟動)
```
MATCH p=(n:Movie {title: "Inception"})-[:DIRECTED|ACTED_IN|IN_GENRE]-(m) RETURN p LIMIT 25
```
<img src="images/cypher_movie_inception.png">

### 對於電影，最簡單的相似度是基於 genres (電影類型) 這個屬性

```
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
WHERE m.title = "Inception"
WITH rec, COLLECT(g.name) AS genres, COUNT(*) AS commonGenres
RETURN rec.title, genres, commonGenres
ORDER BY commonGenres DESC LIMIT 10;
```
<img src="images/cypher_movie_genres_inception.png">

以上方法完全忽略用戶的個性化，因此無法達到千人千面，現在考慮個性化。推薦用戶A沒看過的但和已看過的風格類似的影片，查詢語法：

```
MATCH (u:User {name: "Angelica Rodriguez"})-[r:RATED]->(m:Movie),
      (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
WHERE NOT EXISTS( (u)-[:RATED]->(rec) )
WITH rec, [g.name, COUNT(*)] AS scores
RETURN rec.title AS recommendation, rec.year AS year,
COLLECT(scores) AS scoreComponents,
REDUCE (s=0, x in COLLECT(scores) | s+x[1]) AS score
ORDER BY score DESC LIMIT 10
```

1. MATCH A這個人(Angelica Rodriguez)看的電影關聯到的電影類型(genre)，以此去找rec候選影片。
2. WHERE，表示A沒RATE過，也就是沒看過。
3. WITH，表示看每部推薦候選rec，它對應的genre都出現了幾次(在第一行的MATCH查詢中)，也就是這種類型(genre)的電影A看過多少次。匯集作為scores。因為每部電影rec都可能屬於多個genre，因此後半部分要用一個list[]表達。
4. RETURN，表示要回傳的資料內容，第一個是電影title，第二個電影發行year，第三個把scorers設定為scoreComponent，第四個是對第三個中每項的第二個元素求和為score。
5. ORDER，表示按score降冪排列，取前10名。

<img src="images/cypher_movie_personalized_top_10.png">

上圖表示這些電影預測A可能也會喜歡。

### 多種屬性，加權式計算相似度
以上只使用電影類型 genre 一個屬性，而且是很粗糙的累加計分作為推薦策略，而為求更多種組合(結合多種屬性/特性)，採用加權(weighting)策略，比如，除了電影類型(gener)，我們還可以根據演員/導演等等的相似度來判斷兩個電影的相似度。

```
MATCH (m:Movie) WHERE m.title = "Inception"
MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)

WITH m, rec, COUNT(*) AS gs

OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Actor)-[:ACTED_IN]->(rec)
WITH m, rec, gs, COUNT(a) AS as

OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Director)-[:DIRECTED]->(rec)
WITH m, rec, gs, as, COUNT(d) AS ds

RETURN rec.title AS recommendation, (5*gs)+(3*as)+(4*ds) AS score ORDER BY score DESC LIMIT 100
```

1. MATCH，搜尋全面啟動這部電影，用m表示。
2. MATCH，收集和 m 擁有共同電影分類 genre 的候選電影，以rec代表，同時也收集 genre 用 g 表示。
3. WITH，往下傳遞變數 m, rec, 同時把 g 分類的統計數量以 gs 表示。
4. OPTIONAL MATCH，考慮在 m,rec 的條件下，有交集的演員 a。
5. WITH，往下傳遞變數 m, rec, gs, 並把 a 統計總數以 as 表示。
6. 處理導演相似性。
7. RETURN，最主要看score，把genre, actor, director這三個屬性的計數分別乘以 5, 3, 4 的權重，然後將之相加成為最後的 score。

<img src="images/cypher_movie_recommendation_inception_mul;ti_weight.png">

### 「相似度」計算: Jaccard距離 (Jaccard distance, 雅卡爾)

所謂「相似度」，說法其實並不精確，只能算一個相關程度的計算公式，要成為一個嚴格意義上的相似度距離，則需要滿足眾多條件。比如：穩定、語義清晰等等，接著介紹業界最最簡單的相似度度量：Jaccard相似度。主要用來衡量兩個集合的相似程度。

若A和B是集合，則A和B聯集是有所有A的元素和所有B的元素，而沒有其他元素的集合。A和B的聯集通常寫作"A ∪ B"。形式上：

- x是A ∪ B的元素，若且唯若
- x是A的元素，或
- x是B的元素

舉例：集合 {1,2,3} 和 {2,3,4} 的 聯集 是{1,2,3,4}

A和B的交集寫作"A ∩ B"。形式上：

- x屬於A ∩ B B若且唯若
- x屬於A且x屬於B

舉例：集合 {1,2,3} 和 {2,3,4} 的 交集 是{1,2,3}

<img src="images/cypher_similarity_jaccard.png">

### 以Jaccard相似度查詢與 Inception (全面啟動)的電影分類 (genre) 相似的集合

Jaccard指數是0到1之間的數字，表示兩個集合的相似程度，兩個完全相同集合的Jaccard指數是1。如果兩個集合沒有共同元素，則Jaccard索引為0。藉由將兩個集合的交集的大小除以兩個集合的並集來計算Jaccard相似度，可以計算電影類型集的Jaccard指數，以確定兩部電影的相似程度。

以下是以電影類型為基礎，查詢哪些電影是跟《全面啟動》Jaccard 指數最相似的電影？

```
MATCH (m:Movie {title: "Inception"})-[:IN_GENRE]->(g:Genre)-[:IN_GENRE]-(other:Movie)
WITH m, other, COUNT(g) AS intersection, COLLECT(g.name) AS i
MATCH (m)-[:IN_GENRE]->(mg:Genre)
WITH m,other, intersection,i, COLLECT(mg.name) AS s1
MATCH (other)-[:IN_GENRE]->(og:Genre)
WITH m,other,intersection,i, s1, COLLECT(og.name) AS s2

WITH m,other,intersection,s1,s2

WITH m,other,intersection,s1+[x IN s2 WHERE NOT x IN s1] AS union, s1, s2

RETURN m.title, other.title, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC LIMIT 100
```
1. 查詢出電影Inception和與它類型相關性的電影集 other
2. count(g) 其實就是電影Inception和電影集other 的流派交集的數量（共同類型）
3. s1+[x IN s2 WHERE NOT x IN s1] AS union  此 union 即是s1 和 s2 的並集（集合s1 加上 s2中不包含s1 的那部分）
4. ((1.0*intersection)/SIZE(union)) AS jaccard  根據上面的Jaccard指數公式計算所得的指數(index)


<img src="images/cypher_movie_recommendation_Jaccard_distance.png">

### 將 genre, actor, director 這些條件都加進來計算距離

```
MATCH (m:Movie {title: "Inception"})-[:IN_GENRE|:ACTED_IN|:DIRECTED]-(t)<-[:IN_GENRE|:ACTED_IN|:DIRECTED]-(other:Movie)
WITH m, other, COUNT(t) AS intersection, COLLECT(t.name) AS i
MATCH (m)-[:IN_GENRE|:ACTED_IN|:DIRECTED]-(mt)
WITH m,other, intersection,i, COLLECT(mt.name) AS s1
MATCH (other)-[:IN_GENRE|:ACTED_IN|:DIRECTED]-(ot)
WITH m,other,intersection,i, s1, COLLECT(ot.name) AS s2

WITH m,other,intersection,s1,s2

WITH m,other,intersection,s1+[x IN s2 WHERE NOT x IN s1] AS union, s1, s2

RETURN m.title, other.title, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC LIMIT 100

```
<img src="images/cypher_movie_recommendation_Jaccard_distance_2.png">

## Bonus round

You can skip duplicate Nodes, and set high I/O mode to import database by following command:

```
--skip-duplicate-nodes=true --high-io=true
```